### Read Data into schemaRDD using SQLContext
   A word on sparkSQL... it is shipped with CDH, but not support by Cloudera. There is a bit of flux going that makes code not always backwords compatable. Fortunately, there is high demand for the functionality (meaning we can expect support in the future) and tweeking changes between verions hasn't been overwheling. Given that disclosure, let look at using SQLContext.
   In our example here we are going to read directly from the underlying hdfs folder which contains parquet files. Becuase parquet includes schema, it is read on ingest

In [23]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
dat = sqlContext.parquetFile('/user/cloudera/german_parquet')


Let's have a look at our schema


In [25]:
dat.take(3)

[Row(cred=1, acct_bal=1, dur_cred=18, pay_stat=4, purpose=2, cred_amt=1049, value=1, len_emp=2, install_pc=4, sex_married=2, guarantors=1, dur_addr=4, max_val=2, age=21, concurr=3, typ_aprtmnt=1, no_creds=1, occupation=3, no_dep=1, telephone=1, foreign_wkr=1),
 Row(cred=1, acct_bal=1, dur_cred=9, pay_stat=4, purpose=0, cred_amt=2799, value=1, len_emp=3, install_pc=2, sex_married=3, guarantors=1, dur_addr=2, max_val=1, age=36, concurr=3, typ_aprtmnt=1, no_creds=2, occupation=3, no_dep=2, telephone=1, foreign_wkr=1),
 Row(cred=1, acct_bal=2, dur_cred=12, pay_stat=2, purpose=9, cred_amt=841, value=2, len_emp=4, install_pc=2, sex_married=2, guarantors=1, dur_addr=4, max_val=1, age=23, concurr=3, typ_aprtmnt=1, no_creds=1, occupation=2, no_dep=1, telephone=1, foreign_wkr=1)]

### Data cleaning & formatting
   The German debt data was taken from online: [Analysis of German Credit Data](https://onlinecourses.science.psu.edu/stat857/node/215)
   The data source was fortunately well documented with fully populated records (this will not always be the case). However, there are two modifications that we will consistently need to make to data to prepare it for MLlib functions:
#### Modify field data and type
   More specifically make sure that the fields are converted into a form that is expected by MLlib. In our example we will be running a classification decision tree where many of the fields are categorical. The standard form expected by mllib.tree is that the index starts at zero. Further we need to be sure that fields consistently map to the same number to ensure that the model is being applied appropriately. Luckily, this category to number mapping was already done for use with a small exception; in some fields the categories increment starting from 1 however, mllib.tree expects them to increment from 0.
#### Modify row type
   This row type, is really to put data into an object that is serializable and performs well. Additionally, the row type inherently identifies which column is considered the response or label for the record, which is necessary information when working with supervised learning algorithms. The row form is called a LabeledPoint. The expected RDD used in MLlib is an RDD of LabeledPoints.

#### A transform function for LabeledPoints
   Since we know that we will frequently want our data to be used in analysis, we can create a since line transofrmation by defining a function that will prepare the data for MLlib. Fortunately, our function is a simple map of rows from one form to another. This might not always be the case, and there could be instances where aggregation is done to data in the form of a reduction. We will not explore that here.


In [26]:
from pyspark.mllib.regression import LabeledPoint
def make_lp(x):
    vals=x.asDict()
    label=vals['cred']
    feats=[vals['acct_bal']-1,
           vals['dur_cred'],
           vals['pay_stat'],
           vals['purpose'],
           vals['cred_amt'],
           vals['value']-1,
           vals['len_emp']-1,
           vals['install_pc']-1,
           vals['sex_married']-1,
           vals['guarantors']-1,
           vals['dur_addr']-1,
           vals['max_val']-1,
           vals['age'],
           vals['concurr']-1,
           vals['typ_aprtmnt']-1,
           vals['no_creds']-1,
           vals['occupation']-1,
           vals['no_dep']-1,
           vals['telephone']-1,
           vals['foreign_wkr']-1]
    return LabeledPoint(label, feats)

# Need to identify the features which are categorical
cfi = {0:4,5:5,6:5,7:4,8:4,9:3,10:4,11:4,13:3,14:3,15:4,16:4,17:2,18:2,19:2}

With our function defined, it is now a simple one liner to convert the entire RDD into an RDD of Labeled Points suitable for MLlib. We will use the map function. Notice, we cache() the change. This will retain a copy of the rdd in lp form into memory. This will make iterative evaluations more performent.

In [27]:
lp = dat.map(make_lp).cache()
lp.take(3)

[LabeledPoint(1.0, [0.0,18.0,4.0,2.0,1049.0,0.0,1.0,3.0,1.0,0.0,3.0,1.0,21.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0]),
 LabeledPoint(1.0, [0.0,9.0,4.0,0.0,2799.0,0.0,2.0,1.0,2.0,0.0,1.0,0.0,36.0,2.0,0.0,1.0,2.0,1.0,0.0,0.0]),
 LabeledPoint(1.0, [1.0,12.0,2.0,9.0,841.0,1.0,3.0,1.0,1.0,0.0,3.0,0.0,23.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0])]

#### Training a Decision Tree Model
   MLlib has enhamcements coming with every version. Be sure to check the online documentation between upgrades for improvements. The documentation for model we will evaluate today can be found in [Apache Spark Documentation](https://spark.apache.org/docs/1.3.0/api/python/pyspark.mllib.html#module-pyspark.mllib.tree). From there we find the following 

Parameters:	
   ***data*** – Training data: RDD of LabeledPoint. Labels are integers {0,1,...,numClasses}.
   ***numClasses*** – Number of classes for classification.
   ***categoricalFeaturesInfo*** – Map from categorical feature index to number of categories. Any feature not in this map is treated as continuous.
   ***impurity*** – Supported values: “entropy” or “gini”
   ***maxDepth*** – Max depth of tree. E.g., depth 0 means 1 leaf node. Depth 1 means 1 internal node + 2 leaf nodes.
   ***maxBins*** – Number of bins used for finding splits at each node.
   ***minInstancesPerNode*** – Min number of instances required at child nodes to create the parent split
   ***minInfoGain*** – Min info gain required to create a split

In [28]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

model = DecisionTree.trainClassifier(lp, numClasses=2,
                                     categoricalFeaturesInfo=cfi,
                                     impurity='gini',
                                     maxDepth=5, 
                                     maxBins=32)

# TODO: add lables to feature variables - BAB

model.toDebugString()

u'DecisionTreeModel classifier of depth 5 with 61 nodes\n  If (feature 0 in {0.0,1.0})\n   If (feature 1 <= 21.0)\n    If (feature 2 <= 1.0)\n     If (feature 3 <= 6.0)\n      If (feature 10 in {0.0,1.0,2.0})\n       Predict: 0.0\n      Else (feature 10 not in {0.0,1.0,2.0})\n       Predict: 0.0\n     Else (feature 3 > 6.0)\n      Predict: 1.0\n    Else (feature 2 > 1.0)\n     If (feature 1 <= 11.0)\n      If (feature 11 in {0.0})\n       Predict: 1.0\n      Else (feature 11 not in {0.0})\n       Predict: 1.0\n     Else (feature 1 > 11.0)\n      If (feature 4 <= 1291.0)\n       Predict: 0.0\n      Else (feature 4 > 1291.0)\n       Predict: 1.0\n   Else (feature 1 > 21.0)\n    If (feature 5 in {0.0,1.0,2.0})\n     If (feature 1 <= 47.0)\n      If (feature 4 <= 2319.0)\n       Predict: 0.0\n      Else (feature 4 > 2319.0)\n       Predict: 0.0\n     Else (feature 1 > 47.0)\n      If (feature 10 in {1.0,2.0,3.0})\n       Predict: 0.0\n      Else (feature 10 not in {1.0,2.0,3.0})\n       Pr

Once a model is fit, it can easily be used to predict against a data set that is already in Labeled Point form. Here is an example below where we just predict againstour training set.

NOTE: We would typically break up our data into train/validate/test sets.
TODO: Enhance demo for proper ds eval

In [29]:
lp.map(lambda x: x.features).take(3)

[DenseVector([0.0, 18.0, 4.0, 2.0, 1049.0, 0.0, 1.0, 3.0, 1.0, 0.0, 3.0, 1.0, 21.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]),
 DenseVector([0.0, 9.0, 4.0, 0.0, 2799.0, 0.0, 2.0, 1.0, 2.0, 0.0, 1.0, 0.0, 36.0, 2.0, 0.0, 1.0, 2.0, 1.0, 0.0, 0.0]),
 DenseVector([1.0, 12.0, 2.0, 9.0, 841.0, 1.0, 3.0, 1.0, 1.0, 0.0, 3.0, 0.0, 23.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])]

In [30]:
rslt = model.predict(lp.map(lambda x: x.features))

In [31]:
rslt.collect()

[0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

TODO: create ROC chart

TODO: create cost model

TODO: read model from external source

TODO: Build methods to compare models